In [1]:
import os
import io
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util
import cv2
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import streamlit as st
from PIL import Image

paths = {
 'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
 'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
 'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models','my_ssd_mobnet'),   
 'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
 }

In [2]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models','my_ssd_mobnet', 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], 'generate_tfrecord.py'), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], 'label_map.pbtxt')
}

In [3]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-51')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [4]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [5]:
#IMAGE_PATH = os.path.join(paths['IMAGE_PATH'], 'test', 'Alto16.jpg')

In [6]:



img = st.file_uploader("Choose an image...", type="jpg")
if img is not None:
    img_resize = img.resize((1280, 720)
    image = Image.open(img_resize)
    st.image(image, caption='Uploaded Image', use_column_width=True)

    st.write("")
    
if st.button('predict'):
                st.write("Result...")
                
image_np = np.array(image)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=.8,
            agnostic_mode=False)

plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
#plt.show()

img_buf = io.BytesIO()
plt.savefig(img_buf, format='jpg', dpi =600)

im = Image.open(img_buf)
st.image(im)

img_buf.close()
                
                
                


        

SyntaxError: invalid syntax (3076737057.py, line 4)